# Unit 3 - missing values
---

1. Find rows with missing values
2. Remove missing values using dropna()  
3. Fill missing values using fillna()
4. Fill missing values using interpolate()
5. A note on slicing - copy()
6. GroupBy()





In [1]:
import pandas as pd
import numpy as np

In [2]:
url = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv'
vacc_df = pd.read_csv(url)

<a id='section1'></a>

`null` / `na` - no value

`NaN` - **N**ot **a** **N**umber - the value is missing. This value will be ignored in calculations such as `.mean()`


### 1. Find rows with missing values

In [ ]:
vacc_df.isnull().sum()

`isnull()` is a pandas function, so either use it on a dataframe or call it through pd

In [ ]:
pd.isnull(vacc_df).sum()

In [ ]:
vacc_df['daily_vaccinations'].notnull().sum()

In [ ]:
vacc_df['daily_vaccinations'].isnull().sum()

`isnan` is a numpy function

In [ ]:
np.isnan(vacc_df['daily_vaccinations']).sum()

### 2. Remove missing values using dropna() 

##### Look at Zimbabwe for example. Zimbabwe contains missing values:

In [ ]:
zimbabwe = vacc_df.loc[vacc_df.location == 'Zimbabwe']
#zimbabwe.head(10)

In [ ]:
zimbabwe['total_vaccinations'].isnull().sum()

In [ ]:
zimbabwe['total_vaccinations'].notnull().sum()

##### We can see the difference between the number of values per row:

In [ ]:
zimbabwe.count()

##### Remove all rows that contain one or more missing values: 

In [ ]:
zimbabwe.dropna()

Note: `dropna()`, like most other functions in the pandas API returns a new DataFrame 
(a copy of the original with changes) as the result, so you should assign it back if you want to see changes:

In [ ]:
zimbabwe.head()

assign it back:

In [ ]:
zimbabwe2 = zimbabwe.dropna()
zimbabwe2

##### Remove all values for a specific column - using `subset`

In [ ]:
zimbabwe.dropna(subset = ['total_vaccinations'])

For more columns:

In [ ]:
zimbabwe.dropna(subset = ['total_vaccinations', 'daily_vaccinations_per_million']).head()

---
>A summary of the functions so far:
>
>* `.isnull()` - display rows that contain missing values
>* `.notnull()` - display rows that don't contain missing values
>* `.dropna()` - Remove rows with missing values according to parameters:
    * `.dropna()` (default) - drops rows if at least one column has NaN
    * `.dropna(subset = ['column_name'])` - drop rows that contain missing values in the subset of column names
    * `.dropna(how='all')` - drops rows only if all of its columns have NaNs
    * `.dropna(thresh = k)` - k how many non-null values you want to keep (k=3 means the row should contain at least 3 non-null values)
    * `.dropna(axis=1)` - drop columns instead of rows
> 

See documnetation [here.](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)

---


### 3. Fill missing values using fillna()

Use `.fillna()` to fill missing dataframe values with:
* Whatever value you choose
* Mean, median, mode

Replace all NaNs with 0s

In [ ]:
vacc_df.fillna(0, inplace = False )
vacc_df

>`inplace = False` is the default. This doesn't change the vacc_df dataframe. 
>
>To change it you need:
>
>`vacc_df.fillna(0 , inplace = True)`
>
>or to assign:
>
>`vacc_df = vacc_df.fillna(0)`
>
>But we won't do that! This is where some **business understanding** comes in: it's not a good idea to fill a column like `total_vaccinations` with 0s. 
>
>See what happens:

In [ ]:
vacc_df.fillna(0).head(10)

So we'll use 0's only for the daily_vaccinations columns, and perhaps for some other columns (which?)

In [ ]:
vacc_df['daily_vaccinations'].fillna(0 , inplace = True)

checkout some of the data to see that it works

In [ ]:
vacc_df.iloc[0:3,[0,2,7]]

Other options - using central measures:

In [55]:
# Using median
vacc_df['daily_vaccinations'].fillna(vacc_df['daily_vaccinations'].median(), inplace=True)
  
# Using mean
#vacc_df['daily_vaccinations'].fillna(vacc_df['daily_vaccinations'].mean(), inplace=True)
  
# Using mode
#vacc_df['daily_vaccinations'].fillna(vacc_df['daily_vaccinations'].mode(), inplace=True)


What about `total_vaccinations`? - there are some `NaN`s there as well:

In [ ]:
vacc_df.iloc[52:62,[0,2,3]]

For the `total_vaccinations` we'll use `ffill` which fills the missing values with first non-missing value that occurs before it.

Yes, `bfill` exists as well. If does what you think it does :-)

In [ ]:
vacc_df[['total_vaccinations']].fillna(method='ffill')[52:62]
#vacc_df['total_vaccinations'][52:62]

The first value for some country might be NaN 

Business understanding: this isn't good enought! We need to aggregate by country!!

Use `groupby()` and `apply`  (This is more advanced and we will return to it shortly)

We will create a new column here, `newTotal` - so we can see the difference in `total_vaccinations`


In [ ]:
vacc_df['newTotal'] = vacc_df.groupby('location')[['total_vaccinations']].apply(lambda x: x.fillna(method='ffill'))
vacc_df.iloc[52:62,[0,2,3,12]]

### 4. Fill missing values using interpolate()

In [ ]:
vacc_df['newTotal2'] = vacc_df['total_vaccinations'].interpolate(method ='linear') 
vacc_df.iloc[52:62,[0,2,3,12, 13]]

---
>A summary of the functions so far:
>
>* `.fillna()` - fill missing values according to parameters:
    * `.fillna('k')`  - with value k, create a new dataframe
    * `.fillna('k', inplace = True)` - with value k, into the existing dataframe
    * `.fillna(method='ffill')` - fill with first non-missing value that occurs before it 
    * `.fillna(method='bfill')` - fill with first non-missing value that occurs after it  
> * `interpolate` - fill using some interpolation technique
>
>See documnetation:
>
>* [Missing data handling documentation](https://pandas-docs.github.io/pandas-docs-travis/reference/frame.html#missing-data-handling)
---

### 5. A note on slicing

Slicing is taking only part of a dataframe. For example - the slice we named zimbabwe:

In [ ]:
zimbabwe = vacc_df.loc[vacc_df.location == 'Zimbabwe']

When we change data in a slice, we are changing the ORIGINAL dataframe. This will cause a warning to appear:

In [ ]:
zimbabwe.fillna(0, inplace=True)

The warning will disappear if you rerun the command, but it can still be scary. Best way to avoid it is to create a `copy` of the dataframe:

In [ ]:
zimbabwe = vacc_df.loc[vacc_df.location == 'Zimbabwe'].copy()
zimbabwe.fillna(0, inplace=True)

This works fine, no warnings. But - this won't change the original dataframe (which might be a good thing, if you didn't plan to change it, or a bad thing, if you did)

What about changes in the original dataframe? Your copy will not change.
If you do  want your copy to change, use a shallow copy:

In [ ]:
small_example = pd.Series([1, 2], index=["a", "b"])
small_example

In [ ]:
my_deep_copy = small_example.copy()
my_deep_copy

In [ ]:
my_shallow_copy = small_example.copy(deep=False)
my_shallow_copy

Make a change to the dataframe - where will it appear?

In [ ]:
small_example[0] = -100
small_example

In [ ]:
my_deep_copy

In [ ]:
my_shallow_copy

### 6. Groupby()

#### Group according to something + some columns + some summary statistic

The `mean` of `daily_vaccinations` according to `location`:


In [ ]:
vacc_df.groupby('location')[['daily_vaccinations']].mean()

The same, but for two columns (though as we said, not much business logic for mean value of `total_vaccinations`)

In [ ]:
vacc_df.groupby('location')[['daily_vaccinations', 'total_vaccinations']].mean()

Still the same, but using a lambda function

In [ ]:
vacc_df.groupby('location')[['daily_vaccinations', 'total_vaccinations']].apply(lambda x: x.mean())

`fillna()` is not an aggregation function, so the result is different:

In [ ]:
vacc_df.groupby('location')[['daily_vaccinations']].apply(lambda x: x.fillna(x.mean()))

The same but for two columns:

In [ ]:
vacc_df.groupby('location')[['daily_vaccinations', 'total_vaccinations']].apply(lambda x: x.fillna(x.mean()))

---
>A summary:
>
>* `.copy()` - creates a copy of the slice of the dataframe
>
>* `.copy(deep=False)` - updates to the original dataframe will show in the copy
>
>* `.groupby()` - group according to the columns specified
---